In [1]:
import pandas as pd

def load_data(filepath):
    df = pd.read_csv(filepath)
    return df

df = load_data('../data/BankChurners.csv')


# Preprocessing

### Cleaning out mess

In [2]:
def clean_df(df):
    df = df[df.columns[:-2]]
    df = df.drop(['CLIENTNUM'], axis=1)
    return df

df = clean_df(df)

### Splitting

In [3]:
from sklearn.model_selection import train_test_split
# Train test split
X = df.drop('Attrition_Flag', axis=1)
y = df['Attrition_Flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Rebalancing the target variable (SMOTE)

# AdaBoost Classifier

pipeline creation

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


# Identifying categoricals and numericals
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(exclude=['object', 'category']).columns

# Numerical preprocessing
numerical_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()  
)

# Categorical preprocessing
categorical_pipeline = make_pipeline(
    OneHotEncoder(handle_unknown='ignore')
)

# ColumnTransformer 
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_pipeline, categorical_cols),
        ('num', numerical_pipeline, numerical_cols) 
    ],
    remainder='passthrough'
)


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# A pipeline that includes the above
pipe = make_pipeline(preprocessor, AdaBoostClassifier())
pipe.fit(X_train, y_train)

### Evaluation
-  **Accuracy Printing**: Directly using the .score() method gives you the accuracy on the training and test sets.
-  **ROC AUC Calculation**: ROC_AUC is calculated from probabilities (y_proba), which are obtained using predict_proba() method.
-  **Classification Report**: This report gives you a breakdown of precision, recall, f1-score, and support for each class.
-  **Cross Validation**: Re-running cross-validation on the estimator provides an additional layer of validation, confirming that the parameters selected indeed perform well across different subsets of your training data.

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import cross_validate


# Predictions
y_pred = pipe.predict(X_test)

# R² scoring
print("train_score: ", round(pipe.score(X_train, y_train), 3))
print("test_score: ", round(pipe.score(X_test, y_test), 2))

# ROC_AUC
print("ROC_AUC: ", round(roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]), 3))

# Classification report
report = classification_report(y_test, y_pred)
print(report)

# Cross validation
cv_results = cross_validate(pipe, X_train, y_train, cv=5, scoring='accuracy', return_train_score=True)
print("Mean Test Accuracy:", round(cv_results['test_score'].mean(), 3))
print("Mean Train Accuracy:", round(cv_results['train_score'].mean(), 3))
print("Mean Fit Time:", round(cv_results['fit_time'].mean(), 3))
print("Mean Score Time:", round(cv_results['score_time'].mean(), 3))

## Hyperparameter Tuning
**Best Estimator Usage**: By using grid_search.best_estimator_, you ensure that the predictions and evaluations are performed with the hyperparameter-tuned model. This is essential for getting accurate assessments of model performance.
    

In [ ]:
# Define parameter grid for AdaBoost
param_grid = {
    'adaboostclassifier__n_estimators': [50, 100, 200],
    'adaboostclassifier__learning_rate': [0.01, 0.1, 1.0]
}

In [13]:
from sklearn.model_selection import GridSearchCV

# Create a GridSearchCV object
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

# Use the best estimator to make predictions
y_pred_best = grid_search.predict(X_test)


c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the 

Best parameters: {'adaboostclassifier__learning_rate': 1.0, 'adaboostclassifier__n_estimators': 100}
Best cross-validated score: 0.9629682942247204


In [14]:
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import cross_validate

# Best estimator predictions
y_pred = grid_search.best_estimator_.predict(X_test)
y_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]

# Scoring
print("Train Score: ", round(grid_search.best_estimator_.score(X_train, y_train), 3))
print("Test Score: ", round(grid_search.best_estimator_.score(X_test, y_test), 3))

# Calculate and print ROC AUC
print("ROC_AUC: ", round(roc_auc_score(y_test, y_proba), 3))

# Generate and print classification report
report = classification_report(y_test, y_pred)
print(report)

# Perform cross-validation on the best estimator
cv_results = cross_validate(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='accuracy', return_train_score=True)

# Print mean test and training accuracy, and mean fit and score times
print("Mean Test Accuracy:", round(cv_results['test_score'].mean(), 3))
print("Mean Train Accuracy:", round(cv_results['train_score'].mean(), 3))
print("Mean Fit Time:", round(cv_results['fit_time'].mean(), 3))
print("Mean Score Time:", round(cv_results['score_time'].mean(), 3))


Train Score:  0.971
Test Score:  0.958
ROC_AUC:  0.986
                   precision    recall  f1-score   support

Attrited Customer       0.90      0.83      0.86       327
Existing Customer       0.97      0.98      0.97      1699

         accuracy                           0.96      2026
        macro avg       0.93      0.91      0.92      2026
     weighted avg       0.96      0.96      0.96      2026



c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\Viktor C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the 

Mean Test Accuracy: 0.963
Mean Train Accuracy: 0.971
Mean Fit Time: 1.784
Mean Score Time: 0.047
